In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
from llama_index import (
                ServiceContext,
                LangchainEmbedding,
                OpenAIEmbedding,
            )

In [3]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name = "multi-qa-MiniLM-L6-cos-v1"))

In [27]:
local_path = '/Users/jiaruixu/Downloads/ggml-gpt4all-l13b-snoozy.bin'

In [4]:
from llama_index import ServiceContext
from llama_index.indices.prompt_helper import PromptHelper
from llama_index import LLMPredictor
from langchain.llms import GPT4All
    
model = GPT4All(model=local_path)

llama_model_load: loading model from '/Users/jiaruixu/Downloads/ggml-gpt4all-l13b-snoozy.bin' - please wait ...
llama_model_load: n_vocab = 32000
llama_model_load: n_ctx   = 512
llama_model_load: n_embd  = 5120
llama_model_load: n_mult  = 256
llama_model_load: n_head  = 40
llama_model_load: n_layer = 40
llama_model_load: n_rot   = 128
llama_model_load: f16     = 2
llama_model_load: n_ff    = 13824
llama_model_load: n_parts = 2
llama_model_load: type    = 2
llama_model_load: ggml map size = 7759.83 MB
llama_model_load: ggml ctx size = 101.25 KB
llama_model_load: mem required  = 9807.93 MB (+ 3216.00 MB per state)
llama_model_load: loading tensors from '/Users/jiaruixu/Downloads/ggml-gpt4all-l13b-snoozy.bin'
llama_model_load: model size =  7759.39 MB / num tensors = 363
llama_init_from_file: kv self size  =  800.00 MB


In [24]:
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [25]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [28]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]
# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

llama_model_load: loading model from '/Users/jiaruixu/Downloads/ggml-gpt4all-l13b-snoozy.bin' - please wait ...
llama_model_load: n_vocab = 32000
llama_model_load: n_ctx   = 512
llama_model_load: n_embd  = 5120
llama_model_load: n_mult  = 256
llama_model_load: n_head  = 40
llama_model_load: n_layer = 40
llama_model_load: n_rot   = 128
llama_model_load: f16     = 2
llama_model_load: n_ff    = 13824
llama_model_load: n_parts = 2
llama_model_load: type    = 2
llama_model_load: ggml map size = 7759.83 MB
llama_model_load: ggml ctx size = 101.25 KB
llama_model_load: mem required  = 9807.93 MB (+ 3216.00 MB per state)
llama_model_load: loading tensors from '/Users/jiaruixu/Downloads/ggml-gpt4all-l13b-snoozy.bin'
llama_model_load: model size =  7759.39 MB / num tensors = 363
llama_init_from_file: kv self size  =  800.00 MB


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

llm_chain.run(question)

 The question is about which NFL team won a specific event, but it also asks for information that only exists after the fact of asking this question (the birthdate and name of justin bieber). So let me give you two sets of data to consider before I answer your questions:
1) Super Bowl winners in 20th century - https://en.wikipedia.org/wiki/List_of\_Super_Bowl_champions#Wins_by_decade,

In [18]:
llm = LLMPredictor(llm=model)

prompt_helper = PromptHelper(
    max_input_size=4096,
    num_output=5,
    max_chunk_overlap=20,
    chunk_size_limit=600,
)

service_context = ServiceContext.from_defaults(embed_model=embed_model, llm_predictor=llm)

In [19]:
from llama_index import Document, GPTVectorStoreIndex

In [20]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [21]:
docs = [Document(t) for t in newsgroups_train['data']]

In [22]:
index = GPTVectorStoreIndex.from_documents(docs, service_context=service_context)

KeyboardInterrupt: 

In [ ]:
engine = index.as_query_engine()

In [ ]:
engine.query("how are you")